# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [13]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.dungeon.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState
from core.game_dto.StatData import StatData

player = DungeonPlayerState(
    playerId="BOOK-U0459VJ0PN-6AE1B6584DB90B8D2A17C09B7E8DAEBE",
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    stats = StatData(
        strength = 8,
        dexterity = 1,   
    ),
    skillIds=[0,1],    
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21,100],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="스킬 사용법이 뭐야?")],    
    "dungenon_player": player,
    "target_monster_ids":[1001],
}
await graph.ainvoke(inputs, config=config)
result = await graph.ainvoke({
    "messages":[add_human_message("저 보스몬스터는 뭐야?")]
}, config=config)

result = await graph.ainvoke({
    "messages":[add_human_message("무기 스왑 해줘")],
    "dungenon_player": player
}, config=config)

result = await graph.ainvoke({
    "messages":[add_human_message("인벤토리의 악세사리 뭐 있어?")],
    "dungenon_player": player,
}, config=config)

result["messages"]

총 질문 <Used Abilities>
['USAGE_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-U0459VJ0PN-6AE1B6584DB90B8D2A17C09B7E8DAEBE",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 8,
    "dexterity": 1,
    "intelligence": null,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skillIds": [
    0,
    1
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    },
    "finalDemage": null
  },
  "inventory": [
    0,
    21,
    100
  ]
}
</Heroine Info>

<Current Situation>
    <USAGE_GUIDE>

# Game System Overview

This document summarizes the core game mechanics, including the dungeon-exploration-focused

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-26T09:29:57.068977'}, response_metadata={}, id='dab29a09-ec87-482d-a86f-7dae6a8d8b15'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 고유 스킬을 쓸 수 있어!', additional_kwargs={'created_at': '2025-12-26T09:29:58.500710', 'intent_types': ['USAGE_GUIDE', 'MONSTER_GUIDE']}, response_metadata={}, id='44863004-7087-4d6f-b2d7-573d9fca41f9'),
 HumanMessage(content='저 보스몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-26T09:29:58.502427'}, response_metadata={}, id='acdd13ee-00ea-453d-b397-f2f34041cdf9'),
 AIMessage(content='공포라는 보스 몬스터야! HP 8000에 공격력 100, 속도 350 정도고, 넓은 범위 공격이 강해.', additional_kwargs={'created_at': '2025-12-26T09:29:59.632285', 'intent_types': ['MONSTER_GUIDE']}, response_metadata={}, id='fb005ceb-7009-4493-95b2-4b39dac07ddd'),
 HumanMessage(content='무기 스왑 해줘', additional_kwargs={'created_at': '2025-12-26T09:29:59.633888'}, response_metadata={}, id='aa70a9f6-5292-4a63-8f6b-2d7febbbbd9b'),
 AIMessage(content='

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [3]:
result = await graph.ainvoke(
    {"messages": add_human_message(content="저거 뭐야?"),
     "dungenon_player": player,
     "target_monster_ids":[1000],
     }, config=config
)   

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="사용키좀 알려줘"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]

총 질문 <Used Abilities>
['SMALLTALK', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "BOOK-U0459VJ0PN-6AE1B6584DB90B8D2A17C09B7E8DAEBE",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 8,
    "dexterity": 1,
    "intelligence": null,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skillIds": [
    0,
    1
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    },
    "finalDemage": null
  },
  "inventory": [
    0,
    21,
    100
  ]
}
</Heroine Info>

<Current Situation>

    <MONSTER_GUIDE>

Below is the data specification describing the base stats of monsters
that can appear in dungeons.

This data is u

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-24T17:23:30.821515'}, response_metadata={}, id='b293e8d9-ab49-420f-9867-42be644e6e3b'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 고유 스킬을 쓸 수 있어! 기본 공격은 왼쪽 마우스 버튼이야.', additional_kwargs={'created_at': '2025-12-24T17:23:32.687854', 'intent_types': ['USAGE_GUIDE', 'MONSTER_GUIDE']}, response_metadata={}, id='6ee7fb2a-853a-43dd-ab5d-43226f55e5b5'),
 HumanMessage(content='저 보스몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-24T17:23:32.688934'}, response_metadata={}, id='30d259c7-cf72-484a-8f25-d4911c2a35f3'),
 AIMessage(content='공포라는 보스 몬스터야! HP 8000에 공격력 100, 속도 350 정도고, 넓은 범위 공격이 강해.', additional_kwargs={'created_at': '2025-12-24T17:23:33.951988', 'intent_types': ['MONSTER_GUIDE']}, response_metadata={}, id='b339a4de-a861-428a-9ac0-ad5630059254'),
 HumanMessage(content='무기 스왑 해줘', additional_kwargs={'created_at': '2025-12-24T17:23:33.953508'}, response_metadata={}, id='420e9a78-5aa7-4e82-9fab-72adecb99f59'),


## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [10]:
player = DungeonPlayerState(
    playerId="TEST0",
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    stats=StatData(
        strength=8,
        dexterity=1,
    ),
    skillIds=[0, 1],
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 100],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이벤트 어떻게 깨?"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]

# 전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
# expected_outcome='1번 선택은 기억의 고통을 정면으로 마주해 정신적 타격(instant_damage_medium)을 입지만, 트라우마를 전투 감각으로 전환해 스킬 데미지(skill_damage_increase_all)가 크게 상승한다.\n2번 선택은 죄책감을 회피해 즉각적인 피해는 피하지만, 마음이 무거운 족쇄가 되어 이동속도(move_speed_decrease_all)가 감소해 전투와 탐사에서 기동력이 떨어진다.\n3번 선택은 분노로 정신을 끊어내며 일시적 피해(instant_damage_low)를 받는 대신, 신경이 과도하게 흥분해 공격 속도(attack_speed_increase_all)가 빨라져 근접 교전에 유리해진다.\n4번 선택은 죄책감을 인정하고 상처를 들여다보며 내면이 단단해져 최대 체력(hp_increase_all)이 상승하지만, 기억의 잔향이 스킬 운용을 방해해 일정 시간 스킬 데미지 감소(curse_debuff)를 겪게 된다.') floor=1

총 질문 <Used Abilities>
['EVENT_GUIDE', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "TEST0",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 8,
    "dexterity": 1,
    "intelligence": null,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skillIds": [
    0,
    1
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    },
    "finalDemage": null
  },
  "inventory": [
    0,
    21,
    100
  ]
}
</Heroine Info>

<Current Situation>
    <EVENT_GUIDE>
이벤트방에 입장하지 않아서 정보를 확인할 수 없습니다. 페이몬은 "아직은 아무일 없어보여! 무슨 사건이 일어날 때 말해!" 라는식으로 장난스럽게 답해주세요.
    </EVENT_GUIDE>
    <MONSTER_GUIDE>

Below is the data specification

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-24T17:23:30.821515'}, response_metadata={}, id='b293e8d9-ab49-420f-9867-42be644e6e3b'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 고유 스킬을 쓸 수 있어! 기본 공격은 왼쪽 마우스 버튼이야.', additional_kwargs={'created_at': '2025-12-24T17:23:32.687854', 'intent_types': ['USAGE_GUIDE', 'MONSTER_GUIDE']}, response_metadata={}, id='6ee7fb2a-853a-43dd-ab5d-43226f55e5b5'),
 HumanMessage(content='저 보스몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-24T17:23:32.688934'}, response_metadata={}, id='30d259c7-cf72-484a-8f25-d4911c2a35f3'),
 AIMessage(content='공포라는 보스 몬스터야! HP 8000에 공격력 100, 속도 350 정도고, 넓은 범위 공격이 강해.', additional_kwargs={'created_at': '2025-12-24T17:23:33.951988', 'intent_types': ['MONSTER_GUIDE']}, response_metadata={}, id='b339a4de-a861-428a-9ac0-ad5630059254'),
 HumanMessage(content='무기 스왑 해줘', additional_kwargs={'created_at': '2025-12-24T17:23:33.953508'}, response_metadata={}, id='420e9a78-5aa7-4e82-9fab-72adecb99f59'),


## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [11]:
player = DungeonPlayerState(
    playerId="TEST0",
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    stats=StatData(
        strength=8,
        dexterity=1,
    ),
    skillIds=[0, 1],
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 100],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이 던전에 몬스터 뭐 나와?"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]


총 질문 <Used Abilities>
['DUNGEON_NAVIGATOR', 'MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "TEST0",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 8,
    "dexterity": 1,
    "intelligence": null,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skillIds": [
    0,
    1
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    },
    "finalDemage": null
  },
  "inventory": [
    0,
    21,
    100
  ]
}
</Heroine Info>

<Current Situation>
    <DUNGEON_NAVIGATOR>
        <DUNGEON_MAP>
This information describes the overall structure of the dungeon, including room types, connections, and monster placements.



[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-24T17:23:30.821515'}, response_metadata={}, id='b293e8d9-ab49-420f-9867-42be644e6e3b'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 고유 스킬을 쓸 수 있어! 기본 공격은 왼쪽 마우스 버튼이야.', additional_kwargs={'created_at': '2025-12-24T17:23:32.687854', 'intent_types': ['USAGE_GUIDE', 'MONSTER_GUIDE']}, response_metadata={}, id='6ee7fb2a-853a-43dd-ab5d-43226f55e5b5'),
 HumanMessage(content='저 보스몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-24T17:23:32.688934'}, response_metadata={}, id='30d259c7-cf72-484a-8f25-d4911c2a35f3'),
 AIMessage(content='공포라는 보스 몬스터야! HP 8000에 공격력 100, 속도 350 정도고, 넓은 범위 공격이 강해.', additional_kwargs={'created_at': '2025-12-24T17:23:33.951988', 'intent_types': ['MONSTER_GUIDE']}, response_metadata={}, id='b339a4de-a861-428a-9ac0-ad5630059254'),
 HumanMessage(content='무기 스왑 해줘', additional_kwargs={'created_at': '2025-12-24T17:23:33.953508'}, response_metadata={}, id='420e9a78-5aa7-4e82-9fab-72adecb99f59'),


In [12]:
result = await graph.ainvoke(
    {
        "messages": add_human_message(content="저거 패턴좀 알려줘"),
        "dungenon_player": player,
        'target_monster_ids':[1001]
    },
    config=config,
)
result["messages"]

총 질문 <Used Abilities>
['MONSTER_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "TEST0",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "stats": {
    "hp": 250,
    "moveSpeed": 1.0,
    "attackSpeed": 1.0,
    "cooldownReduction": 1.0,
    "strength": 8,
    "dexterity": 1,
    "intelligence": null,
    "critChance": 20.0,
    "skillDamageMultiplier": 1.0,
    "autoAttackMultiplier": 1.0
  },
  "skillIds": [
    0,
    1
  ],
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    },
    "finalDemage": null
  },
  "inventory": [
    0,
    21,
    100
  ]
}
</Heroine Info>

<Current Situation>
    <MONSTER_GUIDE>

Below is the data specification describing the base stats of monsters
that can appear in dungeons.

This data is used to interpret monster health, movement, attack behavio

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-24T17:23:30.821515'}, response_metadata={}, id='b293e8d9-ab49-420f-9867-42be644e6e3b'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 고유 스킬을 쓸 수 있어! 기본 공격은 왼쪽 마우스 버튼이야.', additional_kwargs={'created_at': '2025-12-24T17:23:32.687854', 'intent_types': ['USAGE_GUIDE', 'MONSTER_GUIDE']}, response_metadata={}, id='6ee7fb2a-853a-43dd-ab5d-43226f55e5b5'),
 HumanMessage(content='저 보스몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-24T17:23:32.688934'}, response_metadata={}, id='30d259c7-cf72-484a-8f25-d4911c2a35f3'),
 AIMessage(content='공포라는 보스 몬스터야! HP 8000에 공격력 100, 속도 350 정도고, 넓은 범위 공격이 강해.', additional_kwargs={'created_at': '2025-12-24T17:23:33.951988', 'intent_types': ['MONSTER_GUIDE']}, response_metadata={}, id='b339a4de-a861-428a-9ac0-ad5630059254'),
 HumanMessage(content='무기 스왑 해줘', additional_kwargs={'created_at': '2025-12-24T17:23:33.953508'}, response_metadata={}, id='420e9a78-5aa7-4e82-9fab-72adecb99f59'),


## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [ ]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?"),
     "dungenon_player": player, },
    config=config
)
result["messages"]

In [ ]:
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="그럼 지금 보스방은 못가?"),
#      "dungenon_player": player, },
#     config=config
# )
result = await graph.ainvoke(
    { "messages": add_human_message(content="보스방울 어떻게 봐?"),
     "dungenon_player": player, },
    config=config
)


In [ ]:
result["messages"][-1]

In [ ]:
from agents.fairy.interaction.fairy_interaction_agent import graph_builder as interaction_builder
interaction_graph = interaction_builder.compile() 
interaction_graph.invoke({
    "messages": add_human_message(content="다음방 확인해"),
    "inventory": [0, 21, 42],
})

# 정령 시연 시나리오 (길드)
## 1) 히로인과 대화시
"히로인의 호감도 얻는법 힌트좀 줘"

## 2) 해금 시나리오 가장 쉬운 단계 해제  후
"로코의 과거 이야기를 다시 듣고 싶어" 

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.guild.fairy_guild_agent import graph_builder as guild_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message

memory = MemorySaver()
guild_graph = guild_graph_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 90,
                    " affection":50, "sanity": 50, "thread_id": "100"}
}
result = await guild_graph.ainvoke(
    { "messages": add_human_message(content="히로인의 호감도 얻는 힌트좀 줘") },
    config=config
)
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="로코의 과거 이야기를 다시 듣고 싶어")},
#     config=config
# )
result["messages"]

In [ ]:
await guild_graph.ainvoke(
    { "messages": add_human_message(content="왜 던전에서 사냥해야해? 이유가 있어?") },
    config=config
)

In [ ]:
await guild_graph.ainvoke(
    { "messages": add_human_message(content="나 이제 뭐 해야돼?") },
    config=config
)

In [ ]:
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 100,
                    " affection":100, "sanity": 100, "thread_id": "100"}
}
await guild_graph.ainvoke(
    { "messages": add_human_message(content="모든 메모리를 해제했엉 이제 나 뭐해야해?") },
    config=config
)